In [ ]:
#cálculo numérico y análisis de datos
import numpy as np
import graphviz
from graphviz import Digraph
#manipulación y análisis de datos (estructuras de datos)
import pandas as pd
labels = ['Pelo', 'Altura', 'Peso', 'Proteccion', 'Quemadura']
quemadura_data = pd.read_csv("arbQuemadura.csv", index_col=0)
quemadura_data.head()

In [ ]:
class Graph:
    def __init__(self, data, children=None) -> None:
        self.data = data
        self.children = None
        self.parent = None
        self.set_children(children)
        self.entropy = None

    def set_children(self, children) -> None:
        self.children = children
        if children is not None:
            for child in children:
                child.parent = self
    def get_children(self) -> list:
        return self.children
    def get_data(self) -> list:
        return self.data
    def get_parent(self) -> list:
        return self.parent
    def equals(self, other) -> bool:
        return self.data == other.data
    def get_entropy(self) -> float:
        return self.entropy
    def set_entropy(self, entropy) -> None:
        self.entropy = entropy

    #FIN? DE CLASE GRAPH

In [114]:
data_quemadura = quemadura_data.copy()
data_quemadura['Pelo'] = np.where(data_quemadura['Pelo'] == 'rubio', 2,
                       np.where(data_quemadura['Pelo'] == 'rojo', 1, 0))
data_quemadura['Altura'] = np.where(data_quemadura['Altura'] == 'alto',                 2, np.where(data_quemadura['Altura'] == 'medio', 1, 0))
data_quemadura['Peso'] = np.where(data_quemadura['Peso'] == 'alto', 2,
                       np.where(data_quemadura['Peso'] == 'medio', 1, 0))
data_quemadura['Proteccion'] = np.where(data_quemadura['Proteccion'] == 'si', 1, 0)
#data_quemadura
#datos = ()
#parent = Graph()
#parent.get_data()
data = []
data = data_quemadura['Pelo'][1],data_quemadura['Altura'][1],data_quemadura['Peso'][1],data_quemadura['Proteccion'][1]
#data_quemadura['Pelo']
pelo = np.unique(data_quemadura['Pelo'])
pelo = pelo[::-1]
altura = np.unique(data_quemadura['Altura'])
altura = altura[::-1]
peso = np.unique(data_quemadura['Peso'])
peso = peso[::-1]
proteccion = np.unique(data_quemadura['Proteccion'])
proteccion = proteccion[::-1]
data2 = max(pelo), max(altura), max(peso), max(proteccion)
parent = Graph(data2, None)
child = []
for i in range(len(pelo)):
    print(pelo[i])
    for a in range(len(altura)):
        for p in range(len(peso)):
            for pr in range(len(proteccion)):
                data = pelo[i], altura[a], peso[p], proteccion[pr]
                child.append(Graph(data, None))
    #child.append((pelo[i], altura[i], peso[i], proteccion[i]))
#parent.set_children(child)
##Hacer mas hijos? cambiar el orden? parece que genera los hijos correcto pero es de los datos que tenemos o de todos los datos?
# posible cambio de orden para dejar pelo al final para que sea el primer hijo en generar

child[4].get_data()

2
1
0


(2, 2, 0, 1)

### calcular la entropia

In [ ]:
def calcular_entropia(p):
    return -np.sum(p*np.log2(p))

### calcular la ganancia

In [ ]:
#calcular la ganancia
def calcular_ganancia(valores, var='Pelo'):
    valores, cuantos = np.unique(quemadura_data[var],return_counts=True)
    #genero una lista para calcular la ganancia
    info_A = []
    
    for e in valores:
        se = np.where(quemadura_data[var]==e)
        #print("tiene se",se)
        #particiona por valor
        se_quemadura = quemadura_data['Quemadura'][quemadura_data.index[se[0]]]
        #print("se_quemadura",se_quemadura)
        #valores por subgrupo
        v_se_quemadura, c_se_quemadura = np.unique(se_quemadura,return_counts=True)
        print("subgrupos ",v_se_quemadura,c_se_quemadura)
        #entropia del valor
        se_entropia = abs(calcular_entropia(c_se_quemadura/sum(c_se_quemadura)))
        print("Entropia: ",se_entropia)
        #probabilidad del valor
        se_size = sum(c_se_quemadura)/len(quemadura_data.index)
        #print("probabilidad valor",se_size)
        #almacenar valores
        info_A += [[se_entropia, se_size]]
        print("info_A",info_A)
    #convertir lista en arreglo
    info_A = np.array(info_A)
    #calcular la entropia de la variable
    info_g = np.sum(info_A[:,0]*info_A[:,1])
    print("info_g",info_g)
    return info_g

## Calcular la entropia del arbol

In [ ]:
valores, cuantos = np.unique(quemadura_data['Quemadura'], return_counts=True)
entropia = calcular_entropia(cuantos/sum(cuantos))
print("Entropia del arbol",entropia)
valores,cuantos

In [ ]:
g = Digraph('grafo')
clase = ['Quema', 'No quema']
for lab in labels[0:4]:
    valores, cuantos = np.unique(quemadura_data[lab], return_counts=True)
    entropia = calcular_entropia(cuantos/sum(cuantos))
    valores,cuantos
    info_A= calcular_ganancia(valores, lab)
    ganancia = entropia - info_A
    print("ganancia de {}".format(lab))
    print('************{}************'.format(lab))
    g.node(lab, label='Clase: {} \\n Entropia: {} \\n Valores: {}'.format(lab, round(entropia, 2), cuantos))
    lb = clase[0] if entropia == 0 else clase[1]
    g.edge(lab, labels[0], label=lb)
g

In [ ]:
#utilizando la librería sklearn

from sklearn import tree
from sklearn import preprocessing

data = pd.DataFrame()
le = preprocessing.LabelEncoder()
var = ["Pelo","Altura","Peso","Proteccion","Quemadura"]

for v in var:
    le.fit(quemadura_data[v])
    data[v] = le.transform(quemadura_data[v])
clf = tree.DecisionTreeClassifier(criterion='entropy',max_depth=4)
t_data = data[data.columns[0:4]]
t_target = data[data.columns[4]]
clf = clf.fit(t_data, t_target)
t_data, t_target,clf
dot_data = tree.export_graphviz(clf, out_file=None,
                         feature_names=list(t_data.columns.values),
                         class_names=["NO Quema", "Se quema"],
                         filled=True, rounded=True,
                         special_characters=True)
graph = graphviz.Source(dot_data)
graph